In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
col_list = ["station","msl", "temp","wddir","rhum","wdsp","rain","month","day","hour","avg_wind_pwr"]
df = pd.read_csv('Jan29_to_Feb26.csv', usecols=col_list)

In [ ]:
def adjusted_r2(y,x,r2):
  adj = 1 - float(len(y)-1)/(len(y)-len(x.columns)-1)*(1 - r2)
  return(adj)

In [ ]:
data = []
stations = df.station.unique()
for station in stations:
  station_data = df[df.station == station]
  # Data splitting.
  x = station_data.iloc[:,1:10]
  y = station_data.iloc[:,-1]

  xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.33,random_state=243)

  # AdaBoost
  ada_reg = AdaBoostRegressor(n_estimators=100)
  AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
           n_estimators=200, random_state=243)
  ada_reg.fit(xtrain,ytrain)
  ypred_ada = ada_reg.predict(xtest)
  mse = mean_squared_error(ytest,ypred_ada)
  r2 = r2_score(ytest,ypred_ada)
  adj = adjusted_r2(ytest,xtest,r2)
  data.append([station,'AdaBoost',mse,np.sqrt(mse),r2,adj])

  # KNN
  knn_regr = KNeighborsRegressor(n_neighbors=5,metric="euclidean",weights="distance").fit(xtrain,ytrain)
  ypred_knn = knn_regr.predict(xtest)
  mse = mean_squared_error(ytest,ypred_knn)
  r2 = r2_score(ytest,ypred_knn)
  adj = adjusted_r2(ytest,xtest,r2)
  data.append([station,'KNN',mse,np.sqrt(mse),r2,adj])

  # MLP
  mlp_regr = MLPRegressor(random_state=1, max_iter=1000, activation = "relu").fit(xtrain, ytrain)
  ypred_mlp = mlp_regr.predict(xtest)
  mse = mean_squared_error(ytest,ypred_mlp)
  r2 = r2_score(ytest,ypred_mlp)
  adj = adjusted_r2(ytest,xtest,r2)
  data.append([station,'MLP',mse,np.sqrt(mse),r2,adj])

  # SVR
  svr_regr = SVR(kernel='linear',gamma='auto',max_iter=4000).fit(xtrain,ytrain)
  ypred_svr = svr_regr.predict(xtest)
  mse = mean_squared_error(ytest,ypred_svr)
  r2 = r2_score(ytest,ypred_svr)
  adj = adjusted_r2(ytest,xtest,r2)
  data.append([station,'SVR',mse,np.sqrt(mse),r2,adj])

  # Linear Regression
  l_regr = LinearRegression().fit(xtrain,ytrain)
  ypred_lnr = l_regr.predict(xtest)
  mse = mean_squared_error(ytest,ypred_lnr)
  r2 = r2_score(ytest,ypred_lnr)
  adj = adjusted_r2(ytest,xtest,r2)
  data.append([station,'Linear Regression',mse,np.sqrt(mse),r2,adj])

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6

In [ ]:
result_df = pd.DataFrame(data, columns = ['Station','Algorithm' ,'MSE','RMSE','R2','Adjusted_R2'])
# Only 23 counties are there in result.
# Markree and Phoenix Park doesn't have values for wind speed and direction.
result_df.to_csv('AllAlgo_for_all_counties_with_R2.csv') 
